In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import sys

[x for x in sys.modules.keys() if "torchcodec" in x]


['torchaudio._torchcodec']

In [16]:
import os

# Check if ffmpeg is installed, if not, install it
if not os.path.exists('/usr/bin/ffmpeg'):
    print("FFmpeg not found. Installing FFmpeg...")
    !apt-get update
    !apt-get install -y ffmpeg
    print("FFmpeg installed successfully.")
else:
    print("FFmpeg is already installed.")

FFmpeg is already installed.


In [18]:
!pip install soundfile


In [32]:
!pip install torchaudio==2.1.2


ERROR: Could not find a version that satisfies the requirement torchaudio==2.1.2 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1, 2.8.0, 2.9.0, 2.9.1, 2.10.0)
ERROR: No matching distribution found for torchaudio==2.1.2


In [19]:
!pip install openai-whisper

In [20]:
import os
import torch
import numpy as np
import soundfile as sf
import torchaudio
#import torchcodec
import whisper
from transformers import AutoTokenizer, AutoModel


In [21]:
# -------------------------------------------------
# Device configuration
# -------------------------------------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [22]:
# -------------------------------------------------
# Load Whisper (Speech → Text)
# -------------------------------------------------
WHISPER_MODEL_SIZE = "base"  # tiny | base | small | medium
whisper_model = whisper.load_model(WHISPER_MODEL_SIZE, device=DEVICE)

In [23]:
# -------------------------------------------------
# Load Text Embedding Model
# -------------------------------------------------
TEXT_MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(TEXT_MODEL_NAME)
text_model = AutoModel.from_pretrained(TEXT_MODEL_NAME).to(DEVICE)
text_model.eval()

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [35]:
import soundfile as sf
import librosa
import torch

TARGET_SR = 16000

def load_and_resample_audio(path):
    """
    Load a WAV file using soundfile, convert to mono, resample to 16 kHz,
    and return a torch tensor waveform.
    """

    # 1. Load audio with soundfile
    waveform, sr = sf.read(path)   # waveform is numpy array

    # 2. Convert stereo → mono
    if waveform.ndim > 1:
        waveform = waveform.mean(axis=1)

    # 3. Resample to TARGET_SR if needed
    if sr != TARGET_SR:
        waveform = librosa.resample(
            waveform,
            orig_sr=sr,
            target_sr=TARGET_SR
        )

    # 4. Convert to torch tensor
    waveform = torch.tensor(waveform).float()

    return waveform, TARGET_SR



In [36]:
path ="/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/03-02-01-01-02-01-22.wav"

waveform, sr = load_and_resample_audio(path)

print("Sample rate:", sr)
print("Waveform shape:", waveform.shape)
print("Tensor type:", waveform.dtype)

Sample rate: 16000
Waveform shape: torch.Size([77945])
Tensor type: torch.float32


In [38]:
TEXT_FEATURE_PATH = "/content/drive/MyDrive/Dissertion/Data/Features/text"
if not os.path.exists(TEXT_FEATURE_PATH):
    os.makedirs(TEXT_FEATURE_PATH)

TEXT_TRANSCRIPT_PATH = "/content/drive/MyDrive/Dissertion/Data/video_transcripts"
if not os.path.exists(TEXT_TRANSCRIPT_PATH):
    os.makedirs(TEXT_TRANSCRIPT_PATH)



# -------------------------------------------------
# Text embedding extraction
# -------------------------------------------------
@torch.no_grad()
def extract_and_save_text_embedding(
    audio_path,
    save_dir=TEXT_FEATURE_PATH
):

  audio_name = os.path.splitext(os.path.basename(audio_path))[0]
  # 1. Load audio


  """if audio.ndim == 2:
    audio = audio.mean(axis=1)

  # Whisper expects 16 kHz
  if sr != 16000:
    audio = whisper.audio.resample(audio, sr, 16000)"""

  #audio, sr = sf.read(audio_path, dtype="float32")

  waveform, sr  = load_and_resample_audio(audio_path)

  # 2. Speech → Text
  result = whisper_model.transcribe(waveform, fp16=False)
  transcript = result["text"].strip()

  if not transcript:
    print(f"Warning: No transcript generated for {audio_path}")
    return

  # 3. Text → Embedding
  inputs = tokenizer(
    transcript,
    return_tensors="pt",
    truncation=True,
    max_length=512
  ).to(DEVICE)

  outputs = text_model(**inputs)

  # CLS token embedding
  text_embedding = outputs.last_hidden_state[:, 0, :].squeeze(0)

  # L2 normalization
  text_embedding = text_embedding / text_embedding.norm()

  # 4. Save
  save_path = os.path.join(save_dir, f"{audio_name}.pt")
  #print(f"Saving {save_path}")

  #save only embeddings
  torch.save(text_embedding.cpu(), save_path)

  #save transcripts to TEXT_TRANSCRIPT_PATH
  transcript_path = os.path.join(TEXT_TRANSCRIPT_PATH, f"{audio_name}.txt")
  with open(transcript_path, "w") as f:
    f.write(transcript)
    f.close()

  #return text_embedding.cpu(), transcript, save_path

In [26]:
RAVDESS_DATA_PATH = "/content/drive/MyDrive/Dissertion/Data/Only_audio"

file_list = []
for root, dirs, files in os.walk(RAVDESS_DATA_PATH):

  for file in files:
        file_list.append(os.path.join(root, file))

print(f"total files found {len(file_list)}")
print(file_list[:5])

total files found 2408
['/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/03-02-01-01-02-01-22.wav', '/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/03-02-02-01-01-01-22.wav', '/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/03-02-01-01-01-02-22.wav', '/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/03-02-04-01-02-01-22.wav', '/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/03-02-03-01-01-02-22.wav']


In [39]:
total_files = len(file_list)
cnt = 0
for vis_file in file_list:
    audio_file = vis_file
    if not os.path.exists(audio_file):
        print(f"Warning: Video file not found: {audio_file}. Skipping.")
        continue
    try:
        extract_and_save_text_embedding(audio_file)
        cnt += 1
        print(f"{cnt}/{total_files}  Embeddings generated for file:{audio_file}")
    except ValueError as e:
        print(f"Error processing {audio_file}: {e}. Skipping.")

1/2408  Embeddings generated for file:/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/03-02-01-01-02-01-22.wav
2/2408  Embeddings generated for file:/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/03-02-02-01-01-01-22.wav
3/2408  Embeddings generated for file:/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/03-02-01-01-01-02-22.wav
4/2408  Embeddings generated for file:/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/03-02-04-01-02-01-22.wav
5/2408  Embeddings generated for file:/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/03-02-03-01-01-02-22.wav
6/2408  Embeddings generated for file:/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/03-02-03-02-02-01-22.wav
7/2408  Embeddings generated for file:/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/03-02-02-02-02-01-22.wav
8/2408  Embeddings generated for file:/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/0

In [40]:
#load embeddings from feature path and show properties of them
import torch
from pathlib import Path

def inspect_embeddings(feature_dir):
    print(f"Total files found: {len(list(Path(feature_dir).rglob('*.pt')))}")

    files = sorted(Path(feature_dir).rglob("*.pt"))[:10]
    print(f"Showing {len(files)} files from: {feature_dir}\n")

    for i, f in enumerate(files, 1):
        obj = torch.load(f)

        if isinstance(obj, dict):
            emb = obj.get("embedding")
            extra = f", keys={list(obj.keys())}"
        else:
            emb = obj
            extra = ""

        if emb is None:
            print(f"{i}. {f.name} -> EMPTY{extra}")
            continue

        shape = tuple(emb.shape)
        dtype = emb.dtype
        norm = float(emb.norm(p=2)) if emb.numel() > 0 else 0.0

        print(f"{i}. {f.name}")
        print(f"   path : {f}")
        print(f"   shape: {shape}")
        print(f"   dtype: {dtype}")
        print(f"   L2   : {norm:.4f}{extra}\n")

In [41]:
inspect_embeddings(TEXT_FEATURE_PATH)

Total files found: 10141
Showing 10 files from: /content/drive/MyDrive/Dissertion/Data/Features/text

1. 01-02-01-01-01-01-01.pt
   path : /content/drive/MyDrive/Dissertion/Data/Features/text/01-02-01-01-01-01-01.pt
   shape: (768,)
   dtype: torch.float32
   L2   : 1.0000

2. 01-02-01-01-01-01-02.pt
   path : /content/drive/MyDrive/Dissertion/Data/Features/text/01-02-01-01-01-01-02.pt
   shape: (768,)
   dtype: torch.float32
   L2   : 1.0000

3. 01-02-01-01-01-01-03.pt
   path : /content/drive/MyDrive/Dissertion/Data/Features/text/01-02-01-01-01-01-03.pt
   shape: (768,)
   dtype: torch.float32
   L2   : 1.0000

4. 01-02-01-01-01-01-04.pt
   path : /content/drive/MyDrive/Dissertion/Data/Features/text/01-02-01-01-01-01-04.pt
   shape: (768,)
   dtype: torch.float32
   L2   : 1.0000

5. 01-02-01-01-01-01-05.pt
   path : /content/drive/MyDrive/Dissertion/Data/Features/text/01-02-01-01-01-01-05.pt
   shape: (768,)
   dtype: torch.float32
   L2   : 1.0000

6. 01-02-01-01-01-01-06.pt
   pat